In [ ]:
import sys
import os
from pathlib import Path

# Add the project root to Python path
cwd = os.getcwd()
print(cwd)
project_root = Path(cwd).parent  # Go up to project root
# Sys.path is a list of dirs that python searches through for importing modules and files
# I'm adding the root to the path so the file can import things as if it were running from root
# This way I can keep my tests organized, but it still acts like I'm running from root
sys.path.insert(0, str(project_root))

import jsbgym
from jsbgym.control_system.evals.power_off_evals import *
from jsbgym.control_system.subsystems.power_off_pid import PowerOffControlSubsystemRawPID, PowerOffControlSubsystemDiffPID

In [ ]:
basic_flight_evaluator = PowerOffEval(custom_tracking_vars=[
    prp.propeller_rpm,
    prp.altitude_sl_ft
])
control_sys = PowerOffControlSubsystemDiffPID()
basic_flight_evaluator.batch_test(
    [110, 90],
    [120, 85, 75, 65, 55],
    [HdgCase.HDG_HOLD],
    control_sys,
    1,
    aircraft=jsbgym.c172x,
)

In [ ]:
# Get batch evaluations
print(basic_flight_evaluator.create_batch_eval(list(range(len(basic_flight_evaluator.cases)))))

In [ ]:
indxs, criteria = basic_flight_evaluator.sort_evals()

In [ ]:
print(indxs)
print(f"number of evals done: {len(indxs)}")
# Sanity check that it is sorted on the criteria correctly
print("Stall/Overspeed, SS kias error, max kias overshoot, exceeded expected time to first contact, max load factor")
for idx, c in enumerate(criteria):
    case_type = basic_flight_evaluator.cases[indxs[idx]]
    print(f"{case_type} : {c} : index {indxs[idx]}")


In [ ]:
# Get the trajectory data and precise evaluation for an individual index for analysis
selected_index = 7
print(basic_flight_evaluator.initial_conditions[selected_index])
eval = basic_flight_evaluator.evals[selected_index]
[print(i) for i in eval.items()]
basic_flight_evaluator.plot_eval(selected_index, control_sys.get_data())

In [ ]:
# Run a single simulation from initial conditions
basic_flight_evaluator.run_single_eval([110], [65], HdgCase.HDG_HOLD, control_sys, aircraft=jsbgym.c172x, render_mode="flightgear")